In [21]:
import pandas as pd
import numpy as np
import pickle

In [22]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [23]:
PROFILE_DIR = "./" 

In [24]:
with open(PROFILE_DIR + "profiling_df_gemm_3090_dimBy1024.pickle", "rb") as in_file:
    df = pickle.load(in_file)

In [25]:
with open(PROFILE_DIR + "error_files_list_gemm_3090_dimBy1024.pickle", "rb") as in_file:
    error_files = pickle.load(in_file)

In [31]:
# what the raw profiling dataset looks like
df.head()

,M,K,N,Kernel Name,Block Size,Grid Size,DRAM Frequency (cycle/second),SM Frequency (cycle/second),Elapsed Cycles (cycle),Memory Throughput (%),DRAM Throughput (%),Duration (nsecond),L1/TEX Cache Throughput (%),L2 Cache Throughput (%),SM Active Cycles (cycle),Compute (SM) Throughput (%),Memory [%] (%),Compute (SM) [%] (%),SM: Pipe Tensor Cycles Active (%),SM: Inst Executed Pipe Lsu (%),SM: Inst Executed (%),SM: Issue Active (%),SM: Mio2rf Writeback Active (%),SM: Pipe Alu Cycles Active (%),SM: Mio Inst Issued (%),SM: Inst Executed Pipe Uniform (%),SM: Pipe Fmaheavy Cycles Active (%),SM: Inst Executed Pipe Adu (%),SM: Pipe Fma Cycles Active (%),SM: Mio Pq Read Cycles Active (%),SM: Mio Pq Write Cycles Active (%),IDC: Request Cycles Active (%),SM: Inst Executed Pipe Cbu Pred On Any (%),SM: Inst Executed Pipe Ipa (%),SM: Inst Executed Pipe Tex (%),SM: Inst Executed Pipe Xu (%),SM: Pipe Fp64 Cycles Active (%),L2: T Sectors (%),L2: Lts2xbar Cycles Active (%),L1: M Xbar2l1tex Read Sectors (%),L2: Xbar2lts Cycles Active (%),L2: T Tag Requests (%),L1: Data Pipe Lsu Wavefronts (%),DRAM: Cycles Active (%),L1: M L1tex2xbar Req Cycles Active (%),L1: Lsu Writeback Active (%),L2: D Sectors (%),DRAM: Dram Sectors (%),L1: Lsuin Requests (%),L1: Data Bank Reads (%),L2: D Sectors Fill Device (%),L1: Data Bank Writes (%),L1: Texin Sm2tex Req Cycles Active (%),L1: Data Pipe Tex Wavefronts (%),L1: F Wavefronts (%),L1: Tex Writeback Active (%),L2: D Atomic Input Cycles Active (%),L2: D Sectors Fill Sysmem (%),sm__sass_thread_inst_executed_op_dfma_pred_on.sum.peak_sustained (inst/cycle),sm__sass_thread_inst_executed_op_ffma_pred_on.sum.peak_sustained (inst/cycle),smsp__sass_thread_inst_executed_op_dfma_pred_on.sum.per_cycle_elapsed (inst/cycle),smsp__sass_thread_inst_executed_op_ffma_pred_on.sum.per_cycle_elapsed (inst/cycle),derived__sm__sass_thread_inst_executed_op_ffma_pred_on_x2 (inst),sm__cycles_elapsed.avg.per_second (cycle/second),dram__bytes.sum.peak_sustained (byte/cycle),dram__cycles_elapsed.avg.per_second (cycle/second),derived__sm__sass_thread_inst_executed_op_dfma_pred_on_x2 (inst),derived__smsp__sass_thread_inst_executed_op_ffma_pred_on_x2 (inst),smsp__cycles_elapsed.avg.per_second (cycle/second),smsp__sass_thread_inst_executed_op_fadd_pred_on.sum.per_cycle_elapsed (inst/cycle),smsp__sass_thread_inst_executed_op_fmul_pred_on.sum.per_cycle_elapsed (inst/cycle),dram__bytes.sum.per_second (byte/second),derived__smsp__sass_thread_inst_executed_op_dfma_pred_on_x2 (inst),smsp__sass_thread_inst_executed_op_dadd_pred_on.sum.per_cycle_elapsed (inst/cycle),smsp__sass_thread_inst_executed_op_dmul_pred_on.sum.per_cycle_elapsed (inst/cycle),Executed Ipc Active (inst/cycle),Executed Ipc Elapsed (inst/cycle),Issue Slots Busy (%),Issued Ipc Active (inst/cycle),sm__inst_issued.max.pct_of_peak_sustained_active (%),SM Busy (%),Tensor (All) (%),Tensor (FP) (%),ALU (%),FMA (%),FP64 (%),Tensor (INT) (%),LSU (%),Uniform (%),ADU (%),CBU (%),FMA (FP16) (%),FP64 (DMMA) (%),FP64 (FP64) (%),TEX (%),XU (%),Memory Throughput (byte/second),Mem Busy (%),Max Bandwidth (%),L1/TEX Hit Rate (%),L2 Compression Success Rate (%),L2 Hit Rate (%),Mem Pipes Busy (%),lts__average_gcomp_input_sector_success_rate.pct (%),lts__t_sectors_srcunit_tex.avg.pct_of_peak_sustained_elapsed (%),lts__t_sectors_srcunit_tex_aperture_peer_lookup_miss.sum (sector),lts__t_sectors_srcunit_tex_aperture_sysmem_lookup_miss.sum (sector),lts__t_sectors_srcunit_tex_lookup_miss.sum (sector),dram__bytes_read.sum (byte),dram__bytes_read.sum.pct_of_peak_sustained_elapsed (%),dram__bytes_read.sum.per_second (byte/second),dram__bytes_write.sum (byte),dram__bytes_write.sum.pct_of_peak_sustained_elapsed (%),dram__bytes_write.sum.per_second (byte/second),l1tex__data_pipe_lsu_wavefronts_mem_shared.sum.pct_of_peak_sustained_elapsed (%),l1tex__m_l1tex2xbar_write_bytes.sum (byte),l1tex__m_l1tex2xbar_write_bytes.sum.pct_of_peak_sustained_elapsed (%),l1tex__m_l1tex2xbar_write_bytes.sum.per_second (byte/second),l1tex__

In [32]:
# example of files that were not able to be profiled
error_files[:5]

['1024_19456_64512.csv',
 '1024_20480_65536.csv',
 '1024_27648_65536.csv',
 '1024_29696_46080.csv',
 '1024_30720_49152.csv']

In [37]:
### important columns for now
key_cols = ['M', 'K', 'N', 'Duration (nsecond)']
#### if we care about more stuff later...
key_cols_more = ['M', 'K', 'N', 'Duration (nsecond)', 'Compute (SM) Throughput (%)', 'Memory Throughput (%)', 'Memory Throughput (byte/second)', 'DRAM Throughput (%)', 'Mem Busy (%)', 'Max Bandwidth (%)', 'Mem Pipes Busy (%)', 'Theoretical Occupancy (%)', 'Achieved Occupancy (%)', 
            'Elapsed Cycles (cycle)', 'SM Active Cycles (cycle)', 'SM: Pipe Tensor Cycles Active (%)',
            'Executed Ipc Active (inst/cycle)', 'Executed Ipc Elapsed (inst/cycle)', 'Issue Slots Busy (%)', 'Issued Ipc Active (inst/cycle)', 'SM Busy (%)', 'Tensor (All) (%)', 'ALU (%)', 'FMA (%)', 'FMA (FP16) (%)', 'FP64 (%)', 'FP64 (DMMA) (%)',  
            'L1/TEX Cache Throughput (%)', 'L2 Cache Throughput (%)', 'L1/TEX Hit Rate (%)', 'L2 Hit Rate (%)',
            'Active Warps Per Scheduler (warp)', 'Eligible Warps Per Scheduler (warp)', 'Theoretical Warps Per Scheduler (warp)', 'GPU Maximum Warps Per Scheduler (warp)', 'Active Warps Per Scheduler (warp)', 'Eligible Warps Per Scheduler (warp)', 'Warp Cycles Per Issued Instruction (cycle)', 'Warp Cycles Per Executed Instruction (cycle)',
            'Avg. Executed Instructions Per Scheduler (inst)', 'Executed Instructions (inst)', 'Avg. Issued Instructions Per Scheduler (inst)', 'Issued Instructions (inst)', 'Registers Per Thread (register/thread)', 'Shared Memory Per Block (byte/block)', 'Shared Memory Configuration Size (byte)', 'Driver Shared Memory Per Block (byte/block)', 'Dynamic Shared Memory Per Block (byte/block)', 'Static Shared Memory Per Block (byte/block)', 'Threads (thread)', 'Avg. Threads Executed (thread)', 'Avg. Predicated-On Threads Executed (thread)',
            'Theoretical Active Warps per SM (warp)', 'Achieved Active Warps Per SM (warp)',
            'L1 Wavefronts Shared Excessive (byte)', 'L2 Theoretical Sectors Global Excessive (byte)', 'Branch Instructions Ratio (%)',
            'SM Frequency (cycle/second)', 'DRAM Frequency (cycle/second)', 
            'Kernel Name', 'Block Size', 'Grid Size']

In [38]:
# shrinking dataset to the important columns
df_key = df[key_cols]

In [39]:
# making the dimenstions (main independent variables for now) and duration column (main dependent variable) an integer to be able to analyze correctly
df_key["M"] = df_key["M"].astype(int)
df_key["K"] = df_key["K"].astype(int)
df_key["N"] = df_key["N"].astype(int)
df_key["Duration (nsecond)"] = df_key["Duration (nsecond)"].apply(lambda x: x.replace(",", "")).astype(int)

/tmp/ipykernel_20159/3136777604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_key["M"] = df_key["M"].astype(int)
/tmp/ipykernel_20159/3136777604.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_key["K"] = df_key["K"].astype(int)
/tmp/ipykernel_20159/3136777604.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [41]:
df_key

,M,K,N,Duration (nsecond)
0,128,128,128,6528
1,128,256,128,8416
2,128,512,128,12128
3,128,1024,128,19360
4,128,2048,128,32672
...,...,...,...,...
15461,65536,512,16384,22485344
15462,65536,1024,16384,41793632
15463,65536,2048,16384,81544960
15464,65536,4096,16384,158803968


In [42]:
with open(PROFILE_DIR + "dims_and_latency_df.pickle", "wb") as out_file:
    pickle.dump(df_key, out_file)